In [ ]:
import numpy as np
import random
from matplotlib import pyplot as plt
import plotly.express as px

<h1> Fish Simulation </h1>

In [ ]:
def length_from_weight(weight):
    return (weight**(1/3.0)) / 2.36068 * random.gauss(1.0, 0.05)

class Fish:
    
    def __init__(self, weight_mean, weight_std, speed_factor_mean, speed_factor_std, min_depth, max_depth):
        self.weight = random.gauss(weight_mean, weight_std)
        self.length = length_from_weight(self.weight)
        self.depth = random.uniform(min_depth, max_depth)
        self.speed = self.length * random.gauss(speed_factor_mean, speed_factor_std)
        self.is_sampled = False
        self.position = [-10, self.depth, 0]
        
    def update_position(self, delta_t):
        delta_x = self.speed * delta_t
        self.position[0] += delta_x
        
    def get_position(self):
        return self.position
        
        
class Camera:
    
    def __init__(self, position, fov_degrees, viewing_angle_degrees):
        self.position = position
        self.fov = fov_degrees * np.pi / 180.0
        self.viewing_angle = viewing_angle_degrees * np.pi / 180.0
        
    @staticmethod
    def gen_p_capture(depth, a=1.5, b=2.5, default_p=1.0):
        if depth < a:
            return default_p
        else:
            return max(default_p * (b - depth) / (b - a), 0)
        
    def contains(self, fish):
        fish_position = fish.get_position()
        fish_segment_at_depth = (fish_position[0] - fish.length / 2.0, fish_position[0] + fish.length / 2.0)
        field_size = fish_position[1] * np.tan(self.fov / 2.0 - self.viewing_angle) + \
                     fish_position[1] * np.tan(self.fov / 2.0 + self.viewing_angle)
#         print(field_size)
        field_center = self.position[0] + fish_position[1] * np.tan(self.viewing_angle)
        field_segment_at_depth = (field_center - field_size / 2.0, field_center + field_size / 2.0)
#         print(field_segment_at_depth)
#         print(fish_segment_at_depth)
        if (fish_segment_at_depth[0] > field_segment_at_depth[0]) and \
            (fish_segment_at_depth[1] < field_segment_at_depth[1]):
            return random.random() < self.gen_p_capture(fish_position[1])
        return False
        


In [ ]:
left_camera = Camera((0, 0, 0), 54, 0)
left_camera.contains(sampled_fishes_54_1_tilt[0])

In [ ]:
left_camera.__dict__

In [ ]:
sampled_fishes_54_1_tilt[0].__dict__

In [ ]:
-.3 - .67/2

In [ ]:
# global fishes


def spawn_fish(fishes):
    fish = Fish(5.0, 1.0, 0.9, 0.05, 0.3, 3.0)
    fishes.append(fish)
    
    
def move_fish(t, t_new, fishes):
    delta_t = t_new - t
    for fish in fishes:
        fish.update_position(delta_t)
        
    fishes = [fish for fish in fishes if fish.get_position()[0] < 10.0]
    return fishes
    
    
def check_if_fully_visible(fish, left_camera, right_camera):
    return left_camera.contains(fish) and right_camera.contains(fish)
    
    
def trigger_capture(fishes, sampled_fishes, left_camera, right_camera):
    for fish in fishes:
        is_visible = check_if_fully_visible(fish, left_camera, right_camera)
        if is_visible:
            fish.is_sampled = True
            sampled_fishes.append(fish)
            
    fishes = [fish for fish in fishes if fish.is_sampled == False]
    return fishes
            



In [ ]:
def generate_samples(FOV, FPS, viewing_angle=0.0):
    fishes = []
    sampled_fishes = []
    left_camera = Camera((0, 0, 0), FOV, viewing_angle)
    right_camera = Camera((0.105, 0, 0), FOV, -viewing_angle)

    capture_times = list(np.arange(0, 100000, 1.0 / FPS))
    fish_spawn_times = list(np.cumsum(np.random.exponential(2, 10000)))

    t = 0
    while len(capture_times) > 0 and len(fish_spawn_times) > 0:
        event_type = np.argmin([capture_times[0], fish_spawn_times[0]])
        if event_type == 0:
            t_new = capture_times[0]
            fishes = move_fish(t, t_new, fishes)
            fishes = trigger_capture(fishes, sampled_fishes, left_camera, right_camera)
            t = t_new
            del capture_times[0]
        elif event_type == 1:
            t_new = fish_spawn_times[0]
            fishes = move_fish(t, t_new, fishes)
            spawn_fish(fishes)
            t = t_new
            del fish_spawn_times[0]

        if len(capture_times) % 100000 == 0:
            print(len(capture_times))
    
    return sampled_fishes


In [ ]:
sampled_fishes[0].__dict__

In [ ]:
def d_max(fov_degrees, tilt_degrees):
    fov = fov_degrees * np.pi / 180.0
    tilt = tilt_degrees * np.pi / 180.0
    return 0.105 / (np.tan(fov/2 + tilt) - np.tan(fov/2 - tilt))

In [ ]:
d_max(54, 5)

In [ ]:
sampled_fishes_54_1 = generate_samples(54, 1.0)

In [ ]:
sampled_fishes_54_1_tilt = generate_samples(54, 1.0, 10.0)

In [ ]:
len(sampled_fishes_54_1)

In [ ]:
len(sampled_fishes_54_1_tilt)

In [ ]:
2027/2894

In [ ]:
2*np.tan(54*np.pi / 180.0 / 2)

In [ ]:
np.tan((44)*np.pi / 180.0 / 2) + np.tan((64)*np.pi / 180.0 / 2)

In [ ]:
fovs = np.arange(54, 70, 2.0)
fps_vals = [0.25, 0.5] + list(np.arange(1.0, 6.0, 1.0))
sampled_fishes_54_1 = generate_samples(54, 1.0)

sample_size_pcts = np.zeros((len(fovs), len(fps_vals)))
for i, fov in enumerate(fovs):
    print(fov)
    for j, fps in enumerate(fps_vals):
        sampled_fishes_fps = generate_samples(fov, fps)
        sample_pct = len(sampled_fishes_fps) / len(sampled_fishes_54_1)
        sample_size_pcts[i, j] = sample_pct

In [ ]:
import seaborn as sns

plt.figure(figsize=(20, 10))
sns.heatmap(sample_size_pcts, annot=True, xticklabels=fps_vals, yticklabels=fovs)
plt.show()

In [ ]:
sampled_fishes_54_2 = generate_samples(54, 2.0)

In [ ]:
sampled_fishes_54_4 = generate_samples(54, 4.0)

In [ ]:
sampled_fishes_66_4 = generate_samples(66, 4.0)

In [ ]:
plt.figure(figsize=(20, 10))
plt.hist([fish.depth for fish in sampled_fishes_54_4], bins=20)
plt.hist([fish.depth for fish in sampled_fishes_54_2], bins=20)
plt.hist([fish.depth for fish in sampled_fishes_54_1], bins=20)

plt.grid()
plt.show()

In [ ]:
sampled_fishes_54_1 = generate_samples(54, 1.0)

In [ ]:
sampled_fishes_58_1 = generate_samples(58, 1.0)

In [ ]:
sampled_fishes_62_1 = generate_samples(62, 1.0)

In [ ]:
sampled_fishes_60_5 = generate_samples(60, 5.0)

In [ ]:
plt.figure(figsize=(20, 10))
plt.hist([fish.depth for fish in sampled_fishes_66_1], bins=20)
plt.hist([fish.depth for fish in sampled_fishes_62_1], bins=20)
plt.hist([fish.depth for fish in sampled_fishes_58_1], bins=20)
plt.hist([fish.depth for fish in sampled_fishes_54_1], bins=20)


plt.grid()
plt.show()

In [ ]:
def get_bar_chart_data(sampled_fishes):
    depths = [fish.depth for fish in sampled_fishes]
    weights = [fish.weight for fish in sampled_fishes]
    depth_ticks = np.arange(0.5, 2.5, 0.1)
    

def plot_bar_chart(sampled_fishes):
    pass
    
    

In [ ]:
np.mean([fish.weight for fish in sampled_fishes_54_1 if 0.8 < fish.depth < 1.5])

In [ ]:
np.mean([fish.weight for fish in sampled_fishes_66_4 if 0.8 < fish.depth < 1.5])

In [ ]:
sampled_fishes_60_5 = generate_samples(60, 5.0)

In [ ]:
len([fish for fish in sampled_fishes_54_1])

In [ ]:
len([fish for fish in sampled_fishes_54_5])

In [ ]:
len([fish for fish in sampled_fishes_60_5])

In [ ]:
5504

In [ ]:
plt.figure(figsize=(20, 10))
plt.hist([fish.depth for fish in sampled_fishes], bins=20)
plt.grid()
plt.show()

In [ ]:
plt.figure(figsize=(20, 10))
plt.hist([fish.depth for fish in sampled_fishes], bins=20)
plt.grid()
plt.show()

In [ ]:
plt.figure(figsize=(20, 10))
plt.hist([fish.depth for fish in sampled_fishes], bins=20)
plt.grid()
plt.show()

In [ ]:
len([fish for fish in sampled_fishes if fish.depth < 1.5])

In [ ]:
sampled_fishes[0].__dict__

In [ ]:
left_camera.contains(sampled_fishes[0])